# 7506 - Trabajo práctico 1


##### Librerías

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [3]:
complete_dataset = pd.read_csv("./datasets/properati_argentina_2021.csv")
df = complete_dataset.copy()

# Analisis Exploratorio del Dataset

**Primeras 5 observaciones**

In [4]:
df.head()

,id,start_date,end_date,created_on,latitud,longitud,place_l2,place_l3,place_l4,place_l5,place_l6,operation,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_currency,property_title
0,XZq2dFGxBiboHY3Bo/tqlw==,2021-09-09,2022-01-27,2021-09-09,-38.957632,-68.105443,Neuquén,NaN,NaN,NaN,NaN,Venta,Casa,6.0,NaN,NaN,206.0,NaN,NaN,Casa - Capital
1,8afoK+yMH/rnnlYiktwgNA==,2021-09-09,2021-10-23,2021-09-09,-34.888878,-56.092368,Montevideo,NaN,NaN,NaN,NaN,Venta,Casa,4.0,3.0,193.0,193.0,390000.0,USD,Casa - Malvín
2,iVaG5rBB47dd1Ya4BMXREA==,2021-09-09,2022-03-09,2021-09-09,-32.488717,-58.257499,Entre Ríos,NaN,NaN,NaN,NaN,Venta,Casa,NaN,NaN,110.0,45.0,47000.0,USD,Casa - Uruguay
3,QE1QVMWKAHWkd0qT0FtPWA==,2021-09-09,2022-01-27,2021-09-09,-34.900165,-54.808453,Maldonado,NaN,NaN,NaN,NaN,Venta,Casa,NaN,3.0,NaN,180.0,400000.0,USD,Casa - El Chorro
4,tJ5lSk7IysG/71OGhacNZA==,2021-09-09,2022-01-27,2021-09-09,-38.954528,-68.041862,Neuquén,NaN,NaN,NaN,NaN,Venta,PH,3.0,2.0,NaN,98.0,102000.0,USD,PH - Villa Farrell


**Ultimas 5 observaciones**

In [5]:
df.tail()

,id,start_date,end_date,created_on,latitud,longitud,place_l2,place_l3,place_l4,place_l5,place_l6,operation,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_currency,property_title
460149,EWHC/xcODFkeEA0+pPE6Rw==,2021-01-05,2021-01-08,2021-01-05,-34.599594,-58.588715,Bs.As. G.B.A. Zona Oeste,Tres de Febrero,NaN,NaN,NaN,Venta,Local comercial,NaN,NaN,113.0,113.0,190000.0,USD,LOCAL COMERCIAL MAS DE 100 MTS. - Ciudad jardín-
460150,txKUq0LjMaORjJQeeqTOfw==,2021-01-05,2021-11-01,2021-01-05,-34.582777,-58.545220,Bs.As. G.B.A. Zona Norte,General San Martín,Villa Lynch,NaN,NaN,Venta,Local comercial,NaN,NaN,11580.0,11580.0,3500000.0,USD,Nave Industrial - Villa Lynch
460151,E0zwHT18DX8AxhU22hXgng==,2021-01-05,2021-03-01,2021-01-05,NaN,NaN,Bs.As. G.B.A. Zona Norte,General San Martín,NaN,NaN,NaN,Venta,Local comercial,NaN,NaN,915.0,915.0,800000.0,USD,importante edificio comercial en tres plantas ...
460152,Ec4Tvy/WX5f6RgfaJf2FHQ==,2021-01-05,2021-11-28,2021-01-05,-33.226234,-60.331848,Santa Fe,Villa Constitución,NaN,NaN,NaN,Alquiler,Local comercial,NaN,NaN,NaN,15.0,NaN,NaN,Local - Villa Constitucion
460153,Y7KBafQ+e5noGh2Pio0xLg==,2021-01-05,9999-12-31,2021-01-05,-26.827938,-65.220708,Tucumán,San Miguel de Tucumán,NaN,NaN,NaN,Venta,Local comercial,1.0,NaN,243.0,133.0,160000.0,USD,Local - San Miguel De Tucumán


Las variables de nuestro dataset actual son: 
id                                  object
start_date                  datetime64[ns]
end_date                    datetime64[ns]
created_on                  datetime64[ns]
latitud                            float64
longitud                           float64
place_l2                            object
place_l3                            object
place_l4                            object
place_l5                            object
place_l6                           float64
operation                           object
property_type                       object
property_rooms                     float64
property_bedrooms                  float64
property_surface_total             float64
property_surface_covered           float64
property_price                     float64
property_currency                   object
property_title                      object
dtype: object


#### Filtrado de Dataset
Como primer paso hay que filtrar las observaciones que su tipo de vivienda sea: Casa, PH o Departamento
Luego filtramos la ubicacion quedandonos unicamente con las propiedades de Capital Federal.
Luego consideraremos solo las que esten cotizadas en dolares
Por último tomamos solo las que sean ventas

In [12]:
df = df[(df.property_type == "Casa") | (df.property_type == "PH") | (df.property_type == "Departamento")]
df = df[(df.place_l2 == "Capital Federal") | (df.place_l3 == "Capital Federal")]
df = df[df.property_currency == "USD"]
df = df[df.operation == "Venta"]
df[["place_l2", "place_l3", "property_currency", "property_type"]]

,place_l2,place_l3,property_currency,property_type
25,Capital Federal,NaN,USD,Casa
42,Capital Federal,Boca,USD,Departamento
43,Capital Federal,Boca,USD,Departamento
49,Capital Federal,Once,USD,Departamento
51,Capital Federal,Boedo,USD,PH
...,...,...,...,...
460072,Capital Federal,Villa del Parque,USD,Departamento
460073,Capital Federal,Villa del Parque,USD,Departamento
460074,Capital Federal,Villa del Parque,USD,Departamento
460075,Capital Federal,Villa del Parque,USD,Departamento


## Análisis de variables

### Tipos de variabes

In [20]:
print(f"Las variables de nuestro dataset actual son: \n{df.dtypes}")


Las variables de nuestro dataset actual son: 
id                                  object
start_date                  datetime64[ns]
end_date                    datetime64[ns]
created_on                  datetime64[ns]
latitud                            float64
longitud                           float64
place_l2                            object
place_l3                            object
place_l4                            object
place_l5                            object
place_l6                           float64
operation                           object
property_type                       object
property_rooms                     float64
property_bedrooms                  float64
property_surface_total             float64
property_surface_covered           float64
property_price                     float64
property_currency                   object
property_title                      object
dtype: object


### Conversion de tipo en las fechas


## TODO: Evitar el coerce y graficar % de NaN

In [19]:
df[["start_date", "end_date", "created_on"]] = df[["start_date", "end_date", "created_on"]].apply(pd.to_datetime,
                                                                                                  errors="coerce")
df.dtypes

id                                  object
start_date                  datetime64[ns]
end_date                    datetime64[ns]
created_on                  datetime64[ns]
latitud                            float64
longitud                           float64
place_l2                            object
place_l3                            object
place_l4                            object
place_l5                            object
place_l6                           float64
operation                           object
property_type                       object
property_rooms                     float64
property_bedrooms                  float64
property_surface_total             float64
property_surface_covered           float64
property_price                     float64
property_currency                   object
property_title                      object
dtype: object

### Analisis de variables cuantitativas

In [18]:
description_quantitatives = df.describe()
quantitative_mode = df[description_quantitatives.columns].mode()
quantitative_mode.rename(index={0: "mode"}, inplace=True)
description_quantitatives = description_quantitatives.append(quantitative_mode)
description_quantitatives = description_quantitatives.append(df.median().rename("median"),
                                                             ignore_index=False)
description_quantitatives

,latitud,longitud,place_l6,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,place_l5
count,90519.000000,90519.000000,0.0,93205.000000,83262.000000,89405.000000,90895.000000,9.424900e+04,NaN
mean,-34.591481,-58.449795,NaN,2.822799,2.014629,109.092970,87.931932,2.262382e+05,NaN
std,0.181719,0.292630,NaN,1.426148,1.129767,980.401976,990.927129,3.186035e+05,NaN
min,-53.788624,-71.471192,NaN,1.000000,-1.000000,10.000000,1.000000,5.000000e+03,NaN
25%,-34.615975,-58.466522,NaN,2.000000,1.000000,45.000000,40.000000,9.500000e+04,NaN
50%,-34.595790,-58.439000,NaN,3.000000,2.000000,65.000000,58.000000,1.439000e+05,NaN
75%,-34.577413,-58.408862,NaN,4.000000,3.000000,105.000000,88.000000,2.370000e+05,NaN
max,-24.219827,-54.133330,NaN,40.000000,70.000000,137000.000000,217000.000000,2.140000e+07,NaN
mode,-34.541250,-58.472966,NaN,3.000000,1.000000,50.000000,40.000000,1.100000e+05,NaN
median,-34.595790,-58.439000,NaN,3.000000,2.000000,65.000000,58.000000,1.439000e+05,NaN


## Análisis de valores faltantes

### Porcentaje de valores faltantes



In [13]:
rows = df.shape[0]
print(df.isna().sum() / rows * 100)

id                            0.000000
start_date                    0.000000
end_date                      5.348598
created_on                    0.000000
latitud                       3.957602
longitud                      3.957602
place_l2                      0.000000
place_l3                      0.442445
place_l4                     96.134707
place_l5                    100.000000
place_l6                    100.000000
operation                     0.000000
property_type                 0.000000
property_rooms                1.107704
property_bedrooms            11.657418
property_surface_total        5.139577
property_surface_covered      3.558658
property_price                0.000000
property_currency             0.000000
property_title                0.000000
dtype: float64


Como podemos observar, las columnas place_l4,place_l5 y place_l6 tienen un porcentaje demasiado alto de valores faltantes (aun solo considerando que sean NaNs) por lo que se descartaran dichas columnas. Por otro lado se descartara la columna id ya que en nuestro caso no sera de utilidad

In [14]:
properties_dataset = df.drop(axis="columns", columns=["id", "place_l4", "place_l5", "place_l6"])
properties_dataset.columns.tolist()

['start_date',
 'end_date',
 'created_on',
 'latitud',
 'longitud',
 'place_l2',
 'place_l3',
 'operation',
 'property_type',
 'property_rooms',
 'property_bedrooms',
 'property_surface_total',
 'property_surface_covered',
 'property_price',
 'property_currency',
 'property_title']

In [15]:
rows, columns = properties_dataset.shape
print(f"Hay una cantidad de {rows} observaciones y {columns} variables")

Hay una cantidad de 94249 observaciones y 16 variables


## Limpieza

## Graficos
